# IMDB Movie Review Dataset

This is a library to download and parse the [IMDB's Large Movie Review Dataset](http://ai.stanford.edu/~amaas/data/sentiment/) dataset and a demo of a transformer based model. The dataset has 25K training, and 25K test dataset, plus 50K unlabeled examples.

It's inspired on [Keras' Text classification with Transformer](https://keras.io/examples/nlp/text_classification_with_transformer/) demo.




## Environment Set Up

Let's set up `go.mod` to use the local copy of GoMLX, so it can be developed jointly the dataset code with the model. That's often how data pre-processing and model code is developed together with experimentation.

If you are not changing code, feel free to simply skip this cell. Or if you used a different directory for you projects, change it below.

Notice the directory `${HOME}/Projects/gomlx` is where the GoMLX code is copied by default in [its Docker](https://hub.docker.com/repository/docker/janpfeifer/gomlx_jupyterlab/general).

In [1]:
!*go mod edit -replace github.com/gomlx/gomlx="${HOME}/Projects/gomlx"

## Data Preparation

### Downloading data files

To download, uncompress and untar to the local directory, simply do the following. Notice if it's already downloaded in the given `--data` directory, it returns immediately.

In [2]:
import (
    "github.com/gomlx/gomlx/examples/imdb"
    "github.com/gomlx/gomlx/ml/data"
)

var flagDataDir = flag.String("data", "~/work/imdb", "Directory to cache downloaded and generated dataset files.")

func AssertNoError(err error) {
    if err != nil {
        log.Fatalf("Failed: %+v", err)
    }
}

func AssertDownloaded() {
    *flagDataDir = data.ReplaceTildeInDir(*flagDataDir)
    if !data.FileExists(*flagDataDir) {
        AssertNoError(os.MkdirAll(*flagDataDir, 0777))
    }

    AssertNoError(imdb.Download(*flagDataDir))
}

%%
AssertDownloaded()

> Loading previously generated preprocessed binary file.
Loaded data from "aclImdb.bin": 100000 examples, 141088 unique tokens, 23727054 tokens in total.


### Sampling some examples

It creates a small dataset and print out some random examples.

It also defines the `DType`, used for all internal representations of the model, and the flag `--max_len` that defines the maximum number of tokens used per observation. This will beused in the modeling later.

In [4]:
import (
    "github.com/gomlx/gomlx/examples/imdb"
)

// DType used for the models.
const DType = shapes.Float32

var (
    flagMaxLen              = flag.Int("max_len", 200, "Maximum number of tokens to take from observation.")
)

func Sample() {
    ds := imdb.NewDataset("Test", imdb.Test, *flagMaxLen, 3, DType, true, nil)
    _, inputs, labels, err := ds.Yield()
    AssertNoError(err)
    labelsData := shapes.CastAsDType(labels[0].Local().Flat(), shapes.Int64).([]int)
    for ii := 0; ii < 3; ii++ {
        fmt.Printf("\nlabel=%v, input=%q\n", labelsData[ii], imdb.InputToString(inputs[0], ii))
    }
    fmt.Println()
}

%% --max_len=200
AssertDownloaded()
Sample()


> Loading previously generated preprocessed binary file.
Loaded data from "aclImdb.bin": 100000 examples, 141088 unique tokens, 23727054 tokens in total.

label=0, input="<START> this is supposed to be based on wilkie collins but the only resemblance it bore to that story were the characters names the time period and the settings if they were going to change the story so thoroughly i don t understand why they needed to keep up the pretense that it came from wilkie collins go read the book it s much better"

label=0, input="<START> a blatant rip off of air bud this movie is really about parents worrying too much about parenting all the foibles of the characters eg adult coaches who dont know the game are taken to the extreme so much so that they are totally unbelievable not funny there is no semblance of reality here folks and you ll not develop sympathy for any of the characters the best thing about the movie is the good looking kits uniforms of the opposing team in the first game the 

## Training

We will create 3 different types of models for this demo: **Bag of Words** (or simply **bow**), **CNNs** and **Transformers**.

### Model Support

We first define here a few componens shared among all models:

* `Normalize` according to flag, works for sequence nodes (shaped `[batch_size, sequence_len, embedding_dim]`) and feature nodes (shaped `[batch_size, embedding_dim]`).
* `EmbedTokensGraph` that embeds the tokens, before being consumed by the models.
* `ReadoutGraph` that takes the embeddings after they are pooled on the sequence axis (so shaped `[batch_size, embedding_dim]` it adds a FNN (feed-forward neural network) with a few layers and convert to the final logits.

We also define the corresponding hyperparameters.

In [9]:
import (
    . "github.com/gomlx/gomlx/graph"
    
    "github.com/gomlx/gomlx/ml/context"
    "github.com/gomlx/gomlx/types/shapes"
)

var (
    flagNormalization    = flag.String("norm", "layer", "Type of normalization to use. Valid values are \"none\", \"batch\", \"layer\".")
    flagDropoutRate      = flag.Float64("dropout", 0.15, "Dropout rate")
    flagMaxVocab            = flag.Int("max_vocab", 20000, "Top most frequent words to consider, the rest is considered unknown.")
    flagTokenEmbeddingSize  = flag.Int("token_embed", 32, "Size of token embedding table. There are ~140K unique tokens")
    flagNumHiddenLayers     = flag.Int("hidden_layers", 2, "Number of output hidden layers, stacked with residual connection.")
    flagNumNodes            = flag.Int("num_nodes", 32, "Number of nodes in output hidden layers.")
    flagWordDropoutRate  = flag.Float64("word_dropout", 0, "Dropout rate for whole words of the input")
)

// Normalize `x` according to `--norm` flag. Works for sequence nodes (rank-3) or plain feature nodes (rank-2).
func Normalize(ctx *context.Context, x *Node) *Node {
    switch *flagNormalization {
    case "layer":
        if x.Rank() == 3 {
            // Normalize sequence.
            return layers.LayerNormalization(ctx, x, -2, -1).
                LearnedOffset(true).LearnedScale(true).ScaleNormalization(true).Done()
        } else {
            // Normalize features only.
            return layers.LayerNormalization(ctx, x, -1).Done()
        }
    case "batch":
        return layers.BatchNormalization(ctx, x, -1).Done()
    case "none":
        return x
    }
    exceptions.Panicf("invalid normalization selected %q -- valid values are batch, layer, none", *flagNormalization)
    return nil
}

// EmbedTokensGraph creates embeddings for tokens and returns them along with the mask of used tokens --
// set to false where padding was used.
func EmbedTokensGraph(ctx *context.Context, tokens *Node) (embed, mask *Node) {
    g := tokens.Graph()
    mask = NotEqual(tokens, ZerosLike(tokens)) // Mask of tokens actually used.

    // The token ids are indexed by frequency. Truncate to the vocabulary size considered, replacing
    // ids higher than that by 0.
    maxVocab := len(imdb.LoadedVocab.ListEntries)
    if maxVocab > *flagMaxVocab {
        maxVocab = *flagMaxVocab
    }

    // Limits tokens to the maxVocab.
    tokens = Where(GreaterOrEqual(tokens, Const(g, maxVocab)),
        MulScalar(OnesLike(tokens), float64(maxVocab-1)),
        tokens)

    // Embed tokens: shape=[batchSize, maxLen, embedDim]
    embed = layers.Embedding(ctx.In("tokens"), tokens, DType, maxVocab, *flagTokenEmbeddingSize)
    embed = Where(mask, embed, ZerosLike(embed))
    
    if *flagWordDropoutRate > 0 {
        dims := embed.Shape().Dimensions[:len(embed.Shape().Dimensions)-1]
        dropoutMask := Ones(g, shapes.Make(DType, dims...))
        dropoutMask = layers.Dropout(ctx, dropoutMask, ConstAsDType(g, DType, *flagWordDropoutRate))
        dropoutMask = ExpandDims(dropoutMask, -1)
        embed = Mul(embed, dropoutMask)
    }    
    return
}

// Activation function used for models.
func Activation(x *Node) *Node {
    return layers.Relu(x)
}

// ReadoutGraph takes the embeddings after they have been pooled on the sequence axis, so shaped `[batch_size, embed_dim]`
// adds a FNN on top and readout the final logit.
func ReadoutGraph(ctx *context.Context, embed *Node) *Node {
    g := embed.Graph()
    var dropoutRate *Node
    if *flagDropoutRate > 0 {
        dropoutRate = ConstAsDType(g, DType, *flagDropoutRate)
    }

    // Output layers.
    for ii := 0; ii < *flagNumHiddenLayers; ii++ {
        ctx := ctx.In(fmt.Sprintf("output_dense_%d", ii))
        residual := embed
        if *flagDropoutRate > 0 {
            embed = layers.Dropout(ctx, embed, dropoutRate)
        }
        //embed = Tanh(embed)
        embed = Activation(embed)
        embed = layers.DenseWithBias(ctx, embed, *flagNumNodes)
        embed = Normalize(ctx, embed)
        if ii > 0 {
            // Add residual connection.
            embed = Add(embed, residual)
        }
    }

    // Final embed layer with dimension 1.
    {
        ctx := ctx.In("readout")
        if *flagDropoutRate > 0 {
            embed = layers.Dropout(ctx, embed, dropoutRate)
        }
        embed = Activation(embed)
        embed = layers.DenseWithBias(ctx, embed, 1)
    }
    return embed
}

%%

### Bag Of Words Model

This is the simples model we are going to train. It's basically a no-op that connects the embedding table we defined above with the `ReadoutGraph` FNN afterwards.

We define here placeholders for our future *CNN* and *Transformer* models.

Finally we test that the shape is correct. To actually train and evaluate we still need to define the training loop, what we do in the following section.

In [12]:
import (
    . "github.com/gomlx/gomlx/graph"
    
    "github.com/gomlx/gomlx/ml/context"
)

var (
    flagModel = flag.String("model", "transformer", "Model type: bow or transformer.")
)

// ModelGraph builds the model for our demo. It returns the logits, not the predictions, which works with most losses.
func ModelGraph(ctx *context.Context, spec any, inputs []*Node) []*Node {
    _ = spec // Not used.
    tokens := inputs[0]
    embed, mask := EmbedTokensGraph(ctx, tokens)

    // Normalization function.
    if *flagModel == "bow" {
        // Bag-Of-Words model doesn't do anything, it's just the embedding table for each token.
    } else if *flagModel == "cnn" {
        embed = Conv1DGraph(ctx, embed, mask)
    } else if *flagModel == "transformer" {
        embed = TransformerGraph(ctx, tokens, embed, mask)
    } else {
        exceptions.Panicf("unknown model type %q, only types \"bow\", \"cnn\" and \"transformer\" are implemented", *flagModel)
    }

    // Sum-up per-token embeddings and do a FNN on the output. From now on, the dimensions are `[batch_dim, embed_dim]`
    // Notice we are not using mask.
    embed = ReduceMax(embed, 1)
    logits := ReadoutGraph(ctx, embed)
    return []*Node{logits}
}

func Conv1DGraph(ctx *context.Context, embed, mask *Node) *Node {
    panic("Not implemented.")
    return nil
}

func TransformerGraph(ctx *context.Context, tokens, embed, mask *Node) *Node {
    panic("Not implemented.")
    return nil
}

%% --model=bow

// Let's test that the logits are coming out with the right shape: we want [batch_size, 10], since there are 10 classes.
AssertDownloaded()
manager := NewManager()
ds := imdb.NewDataset("Test", imdb.Test, *flagMaxLen, 3, DType, true, nil)
_, inputs, _, err := ds.Yield()
AssertNoError(err)

g := manager.NewGraph("test")
ctx := context.NewContext(manager)
logits := ModelGraph(ctx, nil, []*Node{Const(g, inputs[0])})
fmt.Printf("Logits shape for batch_size=%d: %s\n", 3, logits[0].Shape())

> Loading previously generated preprocessed binary file.
Loaded data from "aclImdb.bin": 100000 examples, 141088 unique tokens, 23727054 tokens in total.
Logits shape for batch_size=3: (Float32)[3 1]


### Training Loop

Training loop, with the usual bells and whistles. We also do a quick test with 100 steps to check things are working.

In [16]:
import (
    . "github.com/gomlx/gomlx/graph"
    
    "github.com/gomlx/gomlx/ml/context"
    "github.com/gomlx/gomlx/ml/train"
)

var (
    flagOptimizer        = flag.String("optimizer", "adam", "Optimizer, options: adam or sgd.")
    flagNumSteps         = flag.Int("steps", 5000, "Number of gradient descent steps to perform")
    flagBatchSize        = flag.Int("batch", 32, "Batch size for training")
    flagLearningRate     = flag.Float64("learning_rate", 0.0001, "Initial learning rate.")
    flagL2Regularization = flag.Float64("l2_reg", 0, "L2 regularization on kernels. It doesn't interact well with --batch_norm or with --optimizer=adam.")
    
    flagCheckpoint       = flag.String("checkpoint", "", "Directory save and load checkpoints from. If left empty, no checkpoints are created.")
    flagCheckpointKeep   = flag.Int("checkpoint_keep", 10, "Number of checkpoints to keep, if --checkpoint is set.")
    flagPlots            = flag.Bool("plots", true, "Plots during training: perform periodic evaluations, "+
                                     "save results if --checkpoint is set and draw plots, if in a Jupyter notebook.")
)

func trainModel() {
    // Make sure data is downloaded.
    AssertDownloaded()

    // Manager handles creation of ML computation graphs, accelerator resources, etc.
    manager := NewManager()
    fmt.Printf("Platform: %s\n", manager.Platform())

    // Datasets.
    var trainDS, trainEvalDS, testEvalDS train.Dataset
    trainDS = imdb.NewDataset("train", imdb.Train, *flagMaxLen, *flagBatchSize, DType, true, nil)
    trainEvalDS = imdb.NewDataset("train-eval", imdb.Train, *flagMaxLen, *flagBatchSize, DType, false, nil)
    testEvalDS = imdb.NewDataset("test-eval", imdb.Test, *flagMaxLen, *flagBatchSize, DType, false, nil)

    // Parallelize generation of batches, to prevent the dataset from being a bottleneck to the acceleartor (GPU/TPU).
    trainDS = data.Parallel(trainDS)
    trainEvalDS = data.Parallel(trainEvalDS)
    testEvalDS = data.Parallel(testEvalDS)

    // Metrics we are interested.
    meanAccuracyMetric := metrics.NewMeanBinaryLogitsAccuracy("Mean Accuracy", "#acc")
    movingAccuracyMetric := metrics.NewMovingAverageBinaryLogitsAccuracy("Moving Average Accuracy", "~acc", 0.01)

    // Context holds the variables and hyperparameters for the model.
    ctx := context.NewContext(manager)
    ctx.SetParam(optimizers.LearningRateKey, *flagLearningRate)
    ctx.SetParam(layers.L2RegularizationKey, *flagL2Regularization)

    // Checkpoints saving.
    var checkpoint *checkpoints.Handler
    if *flagCheckpoint != "" {
        var err error
        checkpoint, err = checkpoints.Build(ctx).DirFromBase(*flagCheckpoint, *flagDataDir).Keep(*flagCheckpointKeep).Done()
        AssertNoError(err)
    }

    // Create a train.Trainer: this object will orchestrate running the model, feeding
    // results to the optimizer, evaluating the metrics, etc. (all happens in trainer.TrainStep)
    loss := losses.BinaryCrossentropyLogits
    trainer := train.NewTrainer(
        manager, ctx, ModelGraph, loss,
        optimizers.MustOptimizerByName(*flagOptimizer),
        []metrics.Interface{movingAccuracyMetric}, // trainMetrics
        []metrics.Interface{meanAccuracyMetric})   // evalMetrics

    // Use standard training loop.
    loop := train.NewLoop(trainer)
    commandline.AttachProgressBar(loop) // Attaches a progress bar to the loop.

    // Attach a checkpoint: checkpoint every 1 minute of training.
    if checkpoint != nil {
        period := time.Minute * 1
        train.PeriodicCallback(loop, period, true, "saving checkpoint", 100,
            func(loop *train.Loop, metrics []tensor.Tensor) error {
                fmt.Printf("\n[saving checkpoint@%d] [median train step (ms): %d]\n", loop.LoopStep, loop.MedianTrainStepDuration().Milliseconds())
                return checkpoint.Save()
            })
    }

    // Attach a margaid plots: plot points at exponential steps.
    // The points generated are saved along the checkpoint directory (if one is given).
    if *flagPlots {
        _ = margaid.NewDefault(loop, checkpoint.Dir(), 100, 1.1, trainEvalDS, testEvalDS)
    }

    // Loop for given number of steps.
    _, err := loop.RunSteps(trainDS, *flagNumSteps)
    AssertNoError(err)
    fmt.Printf("\t[Step %d] median train step: %d microseconds\n",
        loop.LoopStep, loop.MedianTrainStepDuration().Microseconds())


    // Report final evaluation.
    fmt.Println()
    err = commandline.ReportEval(trainer, trainEvalDS, testEvalDS)
    AssertNoError(err)
}

%% --model=bow --steps=100 --plots=false
trainModel()


> Loading previously generated preprocessed binary file.
Loaded data from "aclImdb.bin": 100000 examples, 141088 unique tokens, 23727054 tokens in total.
Platform: CUDA
Training (100 steps):  100% [========================================] (49 steps/s) [loss=0.718] [~loss=0.686] [~acc=55.91%]         
	[Step 100] median train step: 883 microseconds

Results on train-eval [Parallelized]:
	Mean Loss (#loss): 0.707
	Mean Accuracy (#acc): 50.00%
Results on test-eval [Parallelized]:
	Mean Loss (#loss): 0.707
	Mean Accuracy (#acc): 50.00%


### Training Bag-Of-Words ("bow") Model

A proper run of our *bow* model.

In [17]:
%% --model=bow --steps=20000 --optimizer=adamw
trainModel()

> Loading previously generated preprocessed binary file.
Loaded data from "aclImdb.bin": 100000 examples, 141088 unique tokens, 23727054 tokens in total.
Platform: CUDA


<path vector-effect="non-scaling-stroke" d="M1.768000e+01,5.230743e+00 L1.365173e+02,5.322910e+00 L2.093971e+02,5.936918e+00 L2.634977e+02,5.353666e+00 L3.073165e+02,1.286629e+02 L3.448334e+02,9.376518e+01 L3.779355e+02,1.461736e+02 L4.078967e+02,1.649938e+02 L4.355033e+02,1.735736e+02 L4.612687e+02,1.766145e+02 L4.855427e+02,1.830699e+02 L5.085726e+02,1.874733e+02 L5.306039e+02,1.908616e+02 L5.518045e+02,1.937690e+02 L5.722912e+02,1.947271e+02 L5.921466e+02,1.976055e+02 L6.114698e+02,1.987769e+02 L6.303266e+02,1.972888e+02 L6.487916e+02,2.012565e+02 L6.669114e+02,2.040316e+02 L6.847383e+02,2.048731e+02 L7.023018e+02,2.065686e+02 L7.196363e+02,2.060225e+02 L7.367580e+02,2.076042e+02 L7.536877e+02,2.085826e+02 L7.704470e+02,2.089806e+02 L7.870558e+02,2.093058e+02 L8.035318e+02,2.097932e+02 L8.198887e+02,2.097571e+02 L8.361372e+02,2.093780e+02 L8.522929e+02,2.104781e+02 L8.663200e+02,2.096669e+02 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,5.292198e+00 L1.365173e+02,5.353666e+00 L2.093971e+02,6.090290e+00 L2.634977e+02,5.200000e+00 L3.073165e+02,1.362967e+02 L3.448334e+02,9.772254e+01 L3.779355e+02,1.569431e+02 L4.078967e+02,1.730579e+02 L4.355033e+02,1.839778e+02 L4.612687e+02,1.890379e+02 L4.855427e+02,1.953831e+02 L5.085726e+02,2.005180e+02 L5.306039e+02,2.040316e+02 L5.518045e+02,2.088178e+02 L5.722912e+02,2.108560e+02 L5.921466e+02,2.151495e+02 L6.114698e+02,2.177380e+02 L6.303266e+02,2.168181e+02 L6.487916e+02,2.224097e+02 L6.669114e+02,2.264730e+02 L6.847383e+02,2.289007e+02 L7.023018e+02,2.318927e+02 L7.196363e+02,2.323522e+02 L7.367580e+02,2.364973e+02 L7.536877e+02,2.385615e+02 L7.704470e+02,2.413964e+02 L7.870558e+02,2.440455e+02 L8.035318e+02,2.471023e+02 L8.198887e+02,2.477075e+02 L8.361372e+02,2.509786e+02 L8.522929e+02,2.530955e+02 L8.663200e+02,2.548000e+02 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,4.627053e+01 L1.365173e+02,3.263880e+01 L2.093971e+02,1.662802e+01 L2.634977e+02,2.338888e+01 L3.073165e+02,2.547410e+01 L3.448334e+02,3.752164e+01 L3.779355e+02,5.781325e+01 L4.078967e+02,8.324734e+01 L4.355033e+02,1.061429e+02 L4.612687e+02,1.207399e+02 L4.855427e+02,1.373233e+02 L5.085726e+02,1.421898e+02 L5.306039e+02,1.527413e+02 L5.518045e+02,1.612523e+02 L5.722912e+02,1.669566e+02 L5.921466e+02,1.709211e+02 L6.114698e+02,1.771074e+02 L6.303266e+02,1.883450e+02 L6.487916e+02,1.871794e+02 L6.669114e+02,1.935328e+02 L6.847383e+02,1.986585e+02 L7.023018e+02,2.037629e+02 L7.196363e+02,2.039623e+02 L7.367580e+02,2.099810e+02 L7.536877e+02,2.106877e+02 L7.704470e+02,2.155102e+02 L7.870558e+02,2.211055e+02 L8.035318e+02,2.279073e+02 L8.198887e+02,2.289123e+02 L8.361372e+02,2.300145e+02 L8.522929e+02,2.347873e+02 L8.663200e+02,2.334268e+02 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,0 L1.768000e+01,-6 M1.287025e+02,0 L1.287025e+02,-6 M1.936465e+02,0 L1.936465e+02,-6 M2.397250e+02,0 L2.397250e+02,-6 M2.754663e+02,0 L2.754663e+02,-6 M3.046690e+02,0 L3.046690e+02,-6 M3.293596e+02,0 L3.293596e+02,-6 M3.507475e+02,0 L3.507475e+02,-6 M3.696130e+02,0 L3.696130e+02,-6 M3.864888e+02,0 L3.864888e+02,-6 M4.017547e+02,0 L4.017547e+02,-6 M4.975112e+02,0 L4.975112e+02,-6 M5.624552e+02,0 L5.624552e+02,-6 M6.085337e+02,0 L6.085337e+02,-6 M6.442750e+02,0 L6.442750e+02,-6 M6.734777e+02,0 L6.734777e+02,-6 M6.981683e+02,0 L6.981683e+02,-6 M7.195562e+02,0 L7.195562e+02,-6 M7.384217e+02,0 L7.384217e+02,-6 M7.552975e+02,0 L7.552975e+02,-6 M8.663200e+02,0 L8.663200e+02,-6 "/> 100 200 300 400 500 600 700 800 900 1000 1100 2000 3000 4000 5000 6000 7000 8000 9000 10000 20000 Steps accuracy accuracy metrics Eval on test-eval [Parallelized]: Mean Accuracy Eval on train-eval [Parallelized]: Mean Accuracy Train: Moving Average Accuracy 
 <path vector-effect="non-scaling-stroke" d="M1.768000e+01,2.548000e+02 L1.365173e+02,2.518830e+02 L2.093971e+02,2.511231e+02 L2.634977e+02,2.514230e+02 L3.073165e+02,2.501835e+02 L3.448334e+02,2.489326e+02 L3.779355e+02,2.438323e+02 L4.078967e+02,2.318153e+02 L4.

Training (20000 steps):  100% [========================================] (841 steps/s) [loss=0.246] [~loss=0.229] [~acc=90.57%]         
	[Step 20000] median train step: 786 microseconds

Results on train-eval [Parallelized]:
	Mean Loss (#loss): 0.131
	Mean Accuracy (#acc): 95.76%
Results on test-eval [Parallelized]:
	Mean Loss (#loss): 0.350
	Mean Accuracy (#acc): 85.13%


### CNN Model

Below we redefine our CNN model function, with 2 convolution layers, and train on it.

Notice how well it can overfit to the training data ... but it doesn't help the test results. To improve this one needs some careful regularization.

In [18]:
// Conv1DGraph stacks two 1D convolution layers on top of the sequence embeddings (shaped `[batch_size, sequence_len, embed_dim]`).
func Conv1DGraph(ctx *context.Context, embed, mask *Node) *Node {
    g := embed.Graph()
    // 1D Convolution:
    {
        ctx := ctx.In("conv1")
        embed = Activation(embed)
        embed = layers.Dropout(ctx, embed, ConstAsDType(g, DType, *flagDropoutRate))
        embed = layers.Convolution(ctx, embed).KernelSize(7).Filters(*flagTokenEmbeddingSize).Strides(3).Done()
        embed = Normalize(ctx, embed)
    }
    {
        ctx := ctx.In("conv2")
        embed = Activation(embed)
        embed = layers.Convolution(ctx, embed).KernelSize(7).Filters(*flagTokenEmbeddingSize).Strides(3).Done()
        embed = Normalize(ctx, embed)
    }
    return embed
}

%% --model=cnn --steps=20000 --optimizer=adamw
trainModel()

> Loading previously generated preprocessed binary file.
Loaded data from "aclImdb.bin": 100000 examples, 141088 unique tokens, 23727054 tokens in total.
Platform: CUDA


<path vector-effect="non-scaling-stroke" d="M1.768000e+01,7.249695e+00 L1.365173e+02,6.961713e+00 L2.093971e+02,5.200000e+00 L2.634977e+02,7.163320e+00 L3.073165e+02,7.220920e+00 L3.448334e+02,7.336049e+00 L3.779355e+02,8.341957e+00 L4.078967e+02,6.961713e+00 L4.355033e+02,1.281978e+01 L4.612687e+02,1.502391e+01 L4.855427e+02,1.729860e+01 L5.085726e+02,3.815346e+01 L5.306039e+02,8.439659e+01 L5.518045e+02,1.162342e+02 L5.722912e+02,1.393729e+02 L5.921466e+02,1.517966e+02 L6.114698e+02,1.635818e+02 L6.303266e+02,1.696078e+02 L6.487916e+02,1.746138e+02 L6.669114e+02,1.709435e+02 L6.847383e+02,1.810102e+02 L7.023018e+02,1.833643e+02 L7.196363e+02,1.750474e+02 L7.367580e+02,1.850892e+02 L7.536877e+02,1.856682e+02 L7.704470e+02,1.834702e+02 L7.870558e+02,1.819147e+02 L8.035318e+02,1.854928e+02 L8.198887e+02,1.874169e+02 L8.361372e+02,1.874344e+02 L8.522929e+02,1.859836e+02 L8.663200e+02,1.856156e+02 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,7.278468e+00 L1.365173e+02,7.249695e+00 L2.093971e+02,6.327343e+00 L2.634977e+02,7.278468e+00 L3.073165e+02,7.249695e+00 L3.448334e+02,7.566197e+00 L3.779355e+02,3.333991e+01 L4.078967e+02,8.141009e+00 L4.355033e+02,4.632840e+01 L4.612687e+02,2.827106e+01 L4.855427e+02,4.779736e+01 L5.085726e+02,8.634268e+01 L5.306039e+02,1.360479e+02 L5.518045e+02,1.649771e+02 L5.722912e+02,1.848960e+02 L5.921466e+02,1.885317e+02 L6.114698e+02,2.058804e+02 L6.303266e+02,2.083413e+02 L6.487916e+02,2.189638e+02 L6.669114e+02,2.219240e+02 L6.847383e+02,2.260058e+02 L7.023018e+02,2.338797e+02 L7.196363e+02,2.330201e+02 L7.367580e+02,2.407135e+02 L7.536877e+02,2.440674e+02 L7.704470e+02,2.457030e+02 L7.870558e+02,2.469619e+02 L8.035318e+02,2.499066e+02 L8.198887e+02,2.517202e+02 L8.361372e+02,2.528564e+02 L8.522929e+02,2.540325e+02 L8.663200e+02,2.548000e+02 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,1.929224e+01 L1.365173e+02,2.216516e+01 L2.093971e+02,1.781474e+01 L2.634977e+02,1.933599e+01 L3.073165e+02,2.073318e+01 L3.448334e+02,9.036989e+00 L3.779355e+02,8.828659e+00 L4.078967e+02,1.125722e+01 L4.355033e+02,1.341658e+01 L4.612687e+02,1.934370e+01 L4.855427e+02,2.639070e+01 L5.085726e+02,4.004590e+01 L5.306039e+02,7.598544e+01 L5.518045e+02,1.276456e+02 L5.722912e+02,1.524033e+02 L5.921466e+02,1.729521e+02 L6.114698e+02,1.826706e+02 L6.303266e+02,1.886588e+02 L6.487916e+02,2.022906e+02 L6.669114e+02,2.059001e+02 L6.847383e+02,2.142888e+02 L7.023018e+02,2.177302e+02 L7.196363e+02,2.220671e+02 L7.367580e+02,2.261456e+02 L7.536877e+02,2.276294e+02 L7.704470e+02,2.305487e+02 L7.870558e+02,2.333430e+02 L8.035318e+02,2.368156e+02 L8.198887e+02,2.386001e+02 L8.361372e+02,2.410942e+02 L8.522929e+02,2.447700e+02 L8.663200e+02,2.447960e+02 "/> <path d="M1.768000e+01,0 L1.768000e+01,-6 M1.287025e+02,0 L1.287025e+02,-6 M1.936465e+02,0 L1.936465e+02,-6 M2.397250e+02,0 L2.397250e+02,-6 M2.754663e+02,0 L2.754663e+02,-6 M3.046690e+02,0 L3.046690e+02,-6 M3.293596e+02,0 L3.293596e+02,-6 M3.507475e+02,0 L3.507475e+02,-6 M3.696130e+02,0 L3.696130e+02,-6 M3.864888e+02,0 L3.864888e+02,-6 M4.017547e+02,0 L4.017547e+02,-6 M4.975112e+02,0 L4.975112e+02,-6 M5.624552e+02,0 L5.624552e+02,-6 M6.085337e+02,0 L6.085337e+02,-6 M6.442750e+02,0 L6.442750e+02,-6 M6.734777e+02,0 L6.734777e+02,-6 M6.981683e+02,0 L6.981683e+02,-6 M7.195562e+02,0 L7.195562e+02,-6 M7.384217e+02,0 L7.384217e+02,-6 M7.552975e+02,0 L7.552975e+02,-6 M8.663200e+02,0 L8.663200e+02,-6 " vector-effect="non-scaling-stroke"/> 100 200 300 400 500 600 700 800 900 1000 1100 2000 3000 4000 5000 6000 7000 8000 9000 10000 20000 Steps accuracy accuracy metrics Eval on test-eval [Parallelized]: Mean Accuracy Eval on train-eval [Parallelized]: Mean Accuracy Train: Moving Average Accuracy 
 <path vector-effect="non-scaling-stroke" d="M1.768000e+01,2.548000e+02 L1.365173e+02,2.540590e+02 L2.093971e+02,2.539387e+02 L2.634977e+02,2.541807e+02 L3.073165e+02,2.540658e+02 L3.448334e+02,2.539066e+02 L3.779355e+02,2.538798e+02 L4.078967e+02,2.539164e+02 L4.

Training (20000 steps):  100% [========================================] (514 steps/s) [loss=0.036] [~loss=0.093] [~acc=96.77%]        
	[Step 20000] median train step: 1179 microseconds

Results on train-eval [Parallelized]:
	Mean Loss (#loss): 0.024
	Mean Accuracy (#acc): 99.50%
Results on test-eval [Parallelized]:
	Mean Loss (#loss): 0.615
	Mean Accuracy (#acc): 82.09%


### Transformer Model

Finally a Transformer version of the model. 

Notice it's not significantly better than our previous simple Bag-Of-Words model. Likely because there is not enough data for the transformer to make any difference. The success of transformers in large-language-models is in large part due to the training with huge amounts of unsupervised (or self-supervised) data.


In [19]:
var (
    flagMaxAttLen           = flag.Int("max_att_len", 200, "Maximum attention length: input will be split in ranges of this size.")
    flagNumAttHeads         = flag.Int("att_heads", 2, "Number of attention heads, if --model=transformer.")
    flagNumAttLayers        = flag.Int("att_layers", 1, "Number of stacked attention layers, if --model=transformer.")
    flagAttKeyQueryEmbedDim = flag.Int("att_key_dim", 8, "Dimension of the Key/Query attention embedding.")    
)

// TransformerGraph is the part of the model that takes the word/token embeddings to a tranformed
// embedding through attention ready to be pooled and read out.
func TransformerGraph(ctx *context.Context, input, embed, mask *Node) *Node {
    var newEmbed *Node
    if *flagMaxAttLen >= *flagMaxLen {
        // One transformer window covers the whole length, which makes it trivial.
        newEmbed = TransformerLayers(ctx.In("transformer"), embed, mask)
        embed = Add(embed, newEmbed)
        return embed
    }

    // Split embedding in multiple split embeddings and apply transformer in each of them.
    attLen := *flagMaxAttLen
    sequenceFrom := 0
    for {
        // x.shape = [batchSize, sequence, embedding]
        sequenceTo := sequenceFrom + attLen
        if sequenceTo > *flagMaxLen {
            sequenceTo = *flagMaxLen
            sequenceFrom = sequenceTo - attLen
        }
        // part = x[:, sequenceFrom:sequenceTo, :]
        residual := Slice(embed, AxisRange(), AxisRange(sequenceFrom, sequenceTo), AxisRange())
        partMask := Slice(mask, AxisRange(), AxisRange(sequenceFrom, sequenceTo))
        // Reuse "transformer" scope.
        part := TransformerLayers(ctx.In("transformer").Checked(false), residual, partMask)
        part = Add(residual, part)
        if newEmbed == nil {
            newEmbed = part
        } else {
            newEmbed = Add(newEmbed, part)
        }

        if sequenceTo == *flagMaxLen {
            // Reached end of parts.
            break
        }
        sequenceFrom += attLen - 20 // Attention window overlap 10 tokens among themselves.
    }
    embed = newEmbed // Notice shape changed to `[batchSize, maxAttLen, embedDim]`
    return embed
}

// TransformerLayers builds the stacked transformer layers for the model.
func TransformerLayers(ctx *context.Context, embed, mask *Node) *Node {
    g := embed.Graph()
    shape := embed.Shape()
    embedDim := shape.Dimensions[2]

    var dropoutRate *Node
    if *flagDropoutRate > 0 {
        dropoutRate = ConstAsDType(g, DType, *flagDropoutRate)
    }

    // Create positional embedding variable: it is 1 in every axis, but for the
    // sequence dimension -- there will be one embedding per position.
    // Shape: [1, maxLen, embedDim]
    posEmbedShape := shape.Copy()
    posEmbedShape.Dimensions[0] = 1
    posEmbedVar := ctx.VariableWithShape("positional", posEmbedShape)
    posEmbed := posEmbedVar.ValueGraph(g)
    embed = Add(embed, posEmbed) // Just add the embeddings, seems to work well.

    // Add the requested number of attention layers.
    for ii := 0; ii < *flagNumAttLayers; ii++ {
        // Each layer in its own scope.
        ctx := ctx.In(fmt.Sprintf("AttLayer_%d", ii))
        residual := embed
        embed = layers.MultiHeadAttention(ctx, embed, embed, embed, *flagNumAttHeads, *flagAttKeyQueryEmbedDim).
            SetKeyMask(mask).SetQueryMask(mask).
            SetOutputDim(embedDim).
            SetValueHeadDim(embedDim).Done()
        if *flagDropoutRate > 0 {
            embed = layers.Dropout(ctx.In("dropout_1"), embed, dropoutRate)
        }
        embed = Normalize(ctx.In("normalization_1"), embed)
        attentionOutput := embed

        // Transformers recipe: 2 dense layers after attention.
        embed = layers.Dense(ctx.In("ffn_1"), embed, true, embedDim)
        embed = Tanh(embed)
        embed = layers.Dense(ctx.In("ffn_2"), embed, true, embedDim)
        if *flagDropoutRate > 0 {
            embed = layers.Dropout(ctx.In("dropout_1"), embed, dropoutRate)
        }
        embed = Add(embed, attentionOutput)
        embed = Normalize(ctx.In("normalization_2"), embed)

        // Residual connection: not part of the usual transfomer layer ...
        if ii > 0 {
            embed = Add(residual, embed)
        }
    }
    return embed
}

%% --model=transformer --steps=3000 --batch=100 --optimizer=adamw --max_len=200 --max_att_len=200 --att_heads=4 --att_layers=3 --att_key_dim=32 --max_vocab=10000
trainModel()

> Loading previously generated preprocessed binary file.
Loaded data from "aclImdb.bin": 100000 examples, 141088 unique tokens, 23727054 tokens in total.
Platform: CUDA


100 200 300 400 500 600 700 800 900 1000 1100 2000 3000 Steps accuracy accuracy metrics Eval on test-eval [Parallelized]: Mean Accuracy Eval on train-eval [Parallelized]: Mean Accuracy Train: Moving Average Accuracy 
 100 200 300 400 500 600 700 800 900 1000 1100 2000 3000 Steps 0.200 0.300 0.400 0.500 0.600 loss loss metrics Eval on test-eval [Parallelized]: Mean Loss Eval on train-eval [Parallelized]: Mean Loss Train: Moving Average Loss

Training (3000 steps):  100% [========================================] (9 steps/s) [loss=0.286] [~loss=0.158] [~acc=95.25%]           
	[Step 3000] median train step: 100427 microseconds

Results on train-eval [Parallelized]:
	Mean Loss (#loss): 0.134
	Mean Accuracy (#acc): 96.12%
Results on test-eval [Parallelized]:
	Mean Loss (#loss): 0.401
	Mean Accuracy (#acc): 85.55%
